In [ ]:
# Importação:

import pandas as pd

In [ ]:
# Lendo arquivo de alunos (exportado pelo TopSoft) com os dados já recebidos da API da ActivitySoft:

df_activity = pd.read_json("alunos.json")

# Formatando os nomes para garantir que não excedam 40 caracteres e removendo espaços extras:
df_activity["nome"] = df_activity["nome"].apply(lambda x: x[0:40].strip())

# Ordenando o DataFrame por nome:
df_activity.sort_values(by="nome", inplace=True)
display(df_activity[["nome", "matricula"]])

In [ ]:
# UMA POSSÍVEL SOLUÇÃO PARA LER OS DADOS DO ALUNO DIRETAMENTE DO BANCO DE DADOS

# import sqlite3

# conn = sqlite3.connect('meu_banco.db')
# df_activity = pd.read_sql_table('minha_tabela', conn)

In [ ]:
# Lendo arquivo de carteirinhas (algum já pre-existente ou exportado pelo TopSoft):

with open(r"gi5_cartoes.txt", "r") as file:
    dados = file.read()

linhas = dados.split("\n")
carteirinhas = [{"cartao": i[0:16], "nome": i[16 : 16 + 40]} for i in linhas]

# Criando DataFrame de carteirinhas:
df_cartoes = pd.DataFrame(carteirinhas)

# Formatando os nomes para garantir que não excedam 40 caracteres e removendo espaços extras:
df_cartoes["nome"] = df_cartoes["nome"].apply(lambda x: x[0:40].strip())

# Drop empty names
df_cartoes = df_cartoes[df_cartoes["nome"] != ""]

# Ordenando o DataFrame por nome:
df_cartoes.sort_values(by="nome", inplace=True)
display(df_cartoes)

In [ ]:
# Validando se não existem nomes duplicados:
if df_cartoes["nome"].duplicated().any():
    print("Existem nomes duplicados no DataFrame de carteirinhas.")

# Validando se não existem nomes duplicados:
if df_activity["nome"].duplicated().any():
    print("Existem nomes duplicados no DataFrame de alunos.")

In [ ]:
# Merge dos DataFrames para encontrar os alunos que não possuem carteirinha:

df_merged = pd.merge(df_activity, df_cartoes, left_on="nome", right_on="nome")
display(df_merged[["nome", "cartao"]])

In [ ]:
# Exportando o DataFrame para JSON:
df_merged.to_json("alunos_e_carteirinha.json", orient="records", force_ascii=False)

In [ ]:
# Vamos atualizar o Banco de Dados local com os dados do DataFrame de forma direta:

from topsoft.repository import (
    bind_cartao_acesso_to_aluno,
    get_or_create_cartao_acesso,
    get_aluno_by_matricula,
)

for index, row in df_merged.iterrows():
    cartao_acesso = get_or_create_cartao_acesso(row["cartao"])
    aluno = get_aluno_by_matricula(row["matricula"])

    if aluno:
        # Se o aluno já existe, apenas vincula o cartão de acesso
        bind_cartao_acesso_to_aluno(
            cartao_acesso_id=cartao_acesso.id,
            aluno_id=aluno.id,
        )
        print(f"Cartão {cartao_acesso.numeracao} vinculado ao aluno {aluno.nome}.")
    else:
        print(f"Aluno com matrícula {row['matricula']} não encontrado.")